# Generating a Synthetic Knowledge Dataset Using Generative AI

In this notebook, we will be generating a synthetic dataset that we can specifically use for retrieval augmented generation (RAG) purposes. Specifically, what we will be generating is a corpus of documents representing **knowledge items** that one might find in a typical Fortune 500 companies IT helpdesk context. I will personally be leveraging my Perplexity subscription to generate the data, but you are welcome to use any LLM of your choosing.

## Notebook Setup

In [1]:
# Installing the Python modules in Kaggle
# Note: This cell only needs run once so long as you are persisting the files in your Kaggle notebook. Comment out when no longer needed.
pip_installs = [
    'langchain',
    'langchain-core',
    'langchain-community',
    'langchain_openai'
    'langchain-text-splitters',
    'ragas'
]
from pip_install import perform_pip_install
perform_pip_install(pip_installs)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1

Package langchain installed successfully!
  Using cached langchain_core-0.2.10-py3-none-any.whl.metadata (6.0 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.1.82-py3-none-any.whl.metadata (13 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
  Using cached tenacity-8.4.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached orjson-3.10.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (49 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.18.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.5 kB)
  Using cached typing_ex

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1

Package langchain-core installed successfully!
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.31-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached langchain-0.2.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached langchain_core-0.2.10-py3-none-any.whl.metadata (6.0 kB)
  Using cached langsmith-0.1.82-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.4.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64

...

Package langchain-community installed successfully!


ERROR: Could not find a version that satisfies the requirement langchain_openailangchain-text-splitters (from versions: none)
ERROR: No matching distribution found for langchain_openailangchain-text-splitters


Package langchain_openailangchain-text-splitters failed to install. Error: Command '['/opt/conda/bin/python', '-m', 'pip', 'install', 'langchain_openailangchain-text-splitters', '--target=/kaggle/working/']' returned non-zero exit status 1.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.6 MB/s eta 0:00:00
  Using cached langchain-0.2.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached langchain_core-0.2.10-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_community-0.2.6-py3-none-any.whl.metadata (2.5 kB)
  Using cached pydantic-2.7.4-py3-none-any.whl.metadata (109 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
gcsfs 2024.2.0 requires fsspec==2024.2.0, but you have fsspec 2024.5.0 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.

Package ragas installed successfully!


In [2]:
# Importing the necessary Python libraries
import os
import pandas as pd
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset.generator import TestsetGenerator

In [3]:
# Loading the API key per the appropriate environment
if 'KAGGLE_URL_BASE' in os.environ:
    
    # Loading the API keys from Kaggle secrets
    from load_api_keys import load_api_keys
    API_KEYS = load_api_keys()

In [4]:
# Instantiating the chat model
chat_model = ChatOpenAI(api_key = API_KEYS['PERPLEXITY_API_KEY'],
                        base_url = 'https://api.perplexity.ai',
                        model = 'llama-3-70b-instruct')

## Prompt Engineering

In this section, we'll set up the prompts that we will be using to generate our specific synthetic dataset. Specifically, we are going to need two separate prompts: one to generate the list of knowledge item topics and another to generate a body of text for a knowledge item given a specific knowledge item topic.

In [5]:
# Creating the topic generation prompt
TOPIC_GENERATION_PROMPT = '''You are the head of an IT helpdesk as part of a Fortune 500 company, and your company maintains a knowledge management system with articles used by helpdesk specialists to answer questions related to IT support. It does not matter what kind of Fortune 500 company this is. Each individual knowledge item in the knowledge management system has a title and supporting body of text. An example of a knowledge item title might be "Resetting a User's Password." Please generate a list of 100 example knowledge item titles that might be found in your knowledge management system. Only include the list of the 100 examples. Do not provide any additional commentary. Please do not say something like "Here are the 100 examples."'''

# Creating the prompt to generate the knowledge item body of text
KNOWLEDGE_ITEM_GENERATION_PROMPT = '''You are the head of an IT helpdesk as part of a Fortune 500 company, and your company maintains a knowledge management system with articles used by helpdesk specialists to answer questions related to IT support. It does not matter what kind of Fortune 500 company this is. Each individual knowledge item in the knowledge management system has a title and supporting body of text. Within triple backticks below is an example of a title of one of these knowledge items. Please write a body of text of steps that might be associated to the knowledge article. The body of text should be no longer than 1000 words. Only return the text that would be populated into the body of the knowledge article; do not return any other text.

Knowledge item title:
```
{ki_title}
```
'''

## Generating the Knowledge Item Topics
Now that we have set our prompt engineering appropriately, we're ready to begin using it to generate the list of knowledge item topics. We will be leaning on LangChain to help with this. Specifically, we are going to chain together the prompt engineering, chat model itself, and a special parser that will take the output from the chat model and turn it into a Python list that we can iterate over.

In [6]:
# Instantiating the output parser
output_parser = CommaSeparatedListOutputParser()

# Setting up the chat prompt template
ki_topic_generation_prompt_template = PromptTemplate(
    template = '{ki_topic_generation_prompt}\n{format_instructions}',
    input_variables = ['ki_topic_generation_prompt'],
    partial_variables = {'format_instructions': output_parser.get_format_instructions()}
)

In [7]:
# Chaining the KI topic generation prompt template, chat model, and output parser
ki_topic_generation_chain = ki_topic_generation_prompt_template | chat_model | output_parser

(Note: The following cell has been intentionally commented out for officially saving a new version of the notebook to Kaggle. To use, simply uncomment!)

In [8]:
# # Generating the KI topics list
# ki_topics_list = ki_topic_generation_chain.invoke({
#     'ki_topic_generation_prompt': TOPIC_GENERATION_PROMPT
# })

# # Slimming down the list to ensure there are only a specific amount of topics
# ki_topics_list = ki_topics_list[-100:]

# # Creating a Pandas DataFrame around the topics list
# df_knowledge_items = pd.DataFrame(data = {'ki_topic': ki_topics_list})

# # Prepping to add the KI text
# df_knowledge_items['ki_text'] = ''

# # Saving the knowledge item topics
# df_knowledge_items.to_csv('synthetic_ki_topics.csv', index = False)

## Generating the Knowledge Item Text
Now that we have generated our list of topics, we are ready to produce text per each knowledge item topics. Now, I'm honestly a bit worried about something erroring out as we get into this, so to ensure that we're not re-doing work that we've already done, we're going to be constantly saving this text back to `df_knowledge_items` so that if we have to re-do anything, we'll simply skip over the stuff already completed.

(Note: The following cell has been intentionally commented out for officially saving a new version of the notebook to Kaggle. To use, simply uncomment!)

In [9]:
# Loading the knowledge item topics back in from the checkpoint file
# df_knowledge_items = pd.read_csv('synthetic_ki_topics.csv')
# df_knowledge_items['ki_text'] = ''

In [10]:
# Creating the prompt engineering template to generate the knowledge item text
ki_text_generation_prompt = ChatPromptTemplate.from_messages(messages = [
    HumanMessagePromptTemplate.from_template(template = KNOWLEDGE_ITEM_GENERATION_PROMPT)
])

# Creating the inference chain to generate the knowledge item text
ki_text_chain = ki_text_generation_prompt | chat_model

In [11]:
def generate_ki_text(row):
    '''
    Generates simulated knowledge item text per a given knowledge item topic
    
    Inputs:
        - row (Pandas DataFrame record): A single record from the Pandas DataFrame
        
    Returns:
        - ki_text (str): The knowledge item text generated by the AI model per the record
    '''
    
    # Checking to see if the knowledge item text has already been generated
    if row['ki_text'] == '':
        
        # Generating the knowledge item text
        ki_text = ki_text_chain.invoke({'ki_title': row['ki_topic']}).content
        
        return ki_text
    
    else:
        
        # Returning what is already in place if the string is not empty
        return row['ki_text']

(Note: The following cell has been intentionally commented out for officially saving a new version of the notebook to Kaggle. To use, simply uncomment!)

In [12]:
# # Generating the knowledge item text for any topic that hasn't already been accounted for
# df_knowledge_items['ki_text'] = df_knowledge_items.apply(generate_ki_text, axis = 1)

# # Saving out the final dataset
# df_knowledge_items.to_csv('synthetic_knowledge_items.csv', index = False)

## Alternative Knowledge Item Text
To demonstrate the potential complications around comingling similar knowledge items in the same index, we will be adding a new set of alternative knowledge item text.

In [13]:
# Loading the original data from file
# df_knowledge_items = pd.read_csv('/kaggle/input/synthetic-it-related-knowledge-items/synthetic_knowledge_items.csv')
# df_knowledge_items['alt_ki_text'] = ''

In [14]:
# Creating the prompt to generate the alternative knowledge item body of text
ALT_KNOWLEDGE_ITEM_GENERATION_PROMPT = '''You are the head of an IT helpdesk as part of a fake Fortune 500 company called "Widgetco", and your company maintains a knowledge management system with articles used by helpdesk specialists to answer questions related to IT support. It does not matter what kind of company this is. Each individual knowledge item in the knowledge management system has a title and supporting body of text. Within triple backticks below is an example of a title of one of these knowledge items. Please write a body of text of steps that might be associated to the knowledge article. The body of text should be no longer than 1000 words. Only return the text that would be populated into the body of the knowledge article; do not return any other text. Please also ensure the company name, "Widgetco", is referenced at least once in the body of the knowledge article text.

Knowledge item title:
```
{ki_title}
```
'''

In [15]:
# Creating the prompt engineering template to generate the knowledge item text
alt_ki_text_generation_prompt = ChatPromptTemplate.from_messages(messages = [
    HumanMessagePromptTemplate.from_template(template = ALT_KNOWLEDGE_ITEM_GENERATION_PROMPT)
])

# Creating the inference chain to generate the knowledge item text
alt_ki_text_chain = alt_ki_text_generation_prompt | chat_model

In [16]:
def generate_alt_ki_text(row):
    '''
    Generates simulated alternative knowledge item text per a given knowledge item topic
    
    Inputs:
        - row (Pandas DataFrame record): A single record from the Pandas DataFrame
        
    Returns:
        - alt_ki_text (str): The alternative knowledge item text generated by the AI model per the record
    '''
    
    # Checking to see if the knowledge item text has already been generated
    if row['alt_ki_text'] == '':
        
        # Generating the knowledge item text
        alt_ki_text = alt_ki_text_chain.invoke({'ki_title': row['ki_topic']}).content
        
        return alt_ki_text
    
    else:
        
        # Returning what is already in place if the string is not empty
        return row['alt_ki_text']

(Note: The following cell has been intentionally commented out for officially saving a new version of the notebook to Kaggle. To use, simply uncomment!)

In [17]:
# # Generating the alt knowledge item text for any topic that hasn't already been accounted for
# df_knowledge_items['alt_ki_text'] = df_knowledge_items.apply(generate_alt_ki_text, axis = 1)

# # Saving out the final dataset
# df_knowledge_items.to_csv('synthetic_knowledge_items.csv', index = False)